# IS622 Final Project
### Brian Chu | Dec15, 2015

In [1]:
import os
import sys

# Path for Spark source folder
os.environ['SPARK_HOME']="/home/brian/workspace/cuny_msda_is622/spark-1.5.1-bin-hadoop2.6"

# Append pyspark to Python Path
sys.path.append("/home/brian/workspace/cuny_msda_is622/spark-1.5.1-bin-hadoop2.6/python/")

# Append py4j to Python Path
sys.path.append("/home/brian/workspace/cuny_msda_is622/spark-1.5.1-bin-hadoop2.6/python/lib/py4j-0.8.2.1-src.zip")

# Launch Spark
execfile("/home/brian/workspace/cuny_msda_is622/spark-1.5.1-bin-hadoop2.6/python/pyspark/shell.py")

%qtconsole

In [2]:
# Load required packages
from pyspark.sql import SQLContext
import numpy as np
import pandas as pd
import math

### Dataset: Jester Online Joke Recommender System
**http://eigentaste.berkeley.edu/dataset/**  
Eigentaste: A Constant Time Collaborative Filtering Algorithm. Ken Goldberg, Theresa Roeder, Dhruv Gupta, and Chris Perkins. Information Retrieval, 4(2), 133-151. July 2001.  
  
This dataset includes ratings from [-10, 10] of up to 100 jokes by 24,983 users.

### Load and clean data

In [673]:
jester = sc.textFile("jester2.csv")

# Create RDD, DataFrame, and Pandas objects
jrdd = jester.map(lambda line: line.split(","))
jdf = jrdd.toDF()

Recode missing data (99) to NaN. I found this easier to do in pandas and then reconvert to PySpark dataframe. Perhaps because dataframes are immutable?

In [668]:
# Convert 99 to NaN (in pandas, remake DF)
sqlc = SQLContext(sc)
jpd = jdf.toPandas()
jpd.iloc[:,1:] = jpd.iloc[:,1:].astype(float)
# jpd = jpd.astype(float)
# jpd[jpd==99.] = np.nan
# jpd[jpd==99.] = 0
# jpd = jpd.set_index('_1', drop=True)
# jpd = jpd.astype(float)
jdf = sqlc.createDataFrame(jpd)

In [679]:
jpd = jdf.toPandas()
jpd.iloc[:,1:] = jpd.iloc[:,1:].astype(float)
jpd[jpd==99.] = 0
jpd.iloc[:,1:] = preprocessing.normalize(jpd.iloc[:,1:])
jpd

jdf = sqlc.createDataFrame(jpd)

,_1,_2,_3,_4,_5,_6,_7,_8,_9,_10,...,_92,_93,_94,_95,_96,_97,_98,_99,_100,_101
0,1,-0.12541,0.140966,-0.154918,-0.130863,-0.120599,-0.136315,-0.157965,0.0668746,-0.144013,...,0.0452246,0,0,0,0,0,-0.0902888,0,0,0
1,2,0.0731198,-0.00519724,0.113981,0.0783171,-0.0426532,-0.173122,-0.0130827,-0.095701,0.159143,...,0.0505387,-0.0887116,-0.00519724,0.140863,-0.00340509,-0.0383521,0.0548399,0.00609332,-0.077421,0.019176
2,3,0,0,0,0,0.160645,0.164915,0.160645,0.164915,0,...,0,0,0,0.161535,0,0,0,0,0,0
3,4,0,0.215072,0,0,0.0463628,0.210178,-0.0726351,0.159952,0,...,0,0,0,0.0136513,0,0,0,0,0,0
4,5,0.164344,0.0891326,-0.0806254,-0.104214,0.0262951,0.0309354,0.136116,0.0891326,-0.00850723,...,0.100347,0.107887,0.0825588,0.100347,0.110787,0.0299687,0.0601307,0.126642,0.0348023,0.0309354
5,6,-0.107365,-0.0615998,0.00765647,-0.147909,-0.123374,-0.0751727,-0.151215,-0.0151389,-0.115717,...,-0.0615998,-0.119893,-0.0118327,-0.0515072,-0.0379343,-0.0582936,0.000870054,-0.158002,-0.0878754,-0.0600337
6,7,0,0,0,0,0.20878,-0.239404,0.187634,0.213641,0,...,0,0,0,0,0,0.0566306,0,0,0,0
7,8,0.119504,0.0552093,0.160212,-0.108497,-0.142566,-0.0297012,0.161959,0.0246345,-0.0906761,...,0.126318,-0.0195679,-0.00174713,-0.099237,-0.0552093,-0.0585289,0.0373886,-0.000873565,0.0228874,0
8,9,-0.094115,-0.0879069,-0.233922,-0.171096,-0.217036,-0.00720141,-0.131364,-0.221754,-0.195183,...,0.108518,-0.00720141,0.103551,-0.00720141,-0.00720141,-0.00720141,-0.00720141,-0.00720141,-0.0844304,-0.122921
9,10,0.0685803,0.117338,0.117338,0.0685803,0.146046,0.117338,0.203463,0.0574161,0.0685803,...,0,0.101845,0,0,0,0,0,0,0,0


In [800]:
jdf2 = jrdd.toDF()
jpd2 = jdf2.toPandas()
jpd2.iloc[:,1:] = jpd2.iloc[:,1:].astype(float)
jpd2[jpd2==99.] = 0
jdf2 = sqlc.createDataFrame(jpd2)

from pyspark.mllib.feature import StandardScaler
from pyspark.mllib.feature import Normalizer
normalizer1 = Normalizer()
norm_jdf2 = normalizer1.transform(jdf2)

## Collaborative Filtering Recommendations

### PySpark MLlib  
I am going to use PySpark's MLlib package, which incorporates collaborative filtering using the Alternating Least Squares (ALS) algorithm to predict missing entries. ALS aims to minimize the squared error of the UV user-item rating matrix by alternating learning sequences of U to V and V to U until a steady-state has been reached. This is similar to the collaborative filtering theory discussed in the MMS text.  
  
### Modified source code:  
http://spark.apache.org/docs/latest/mllib-collaborative-filtering.html  
https://databricks-training.s3.amazonaws.com/movie-recommendation-with-mllib.html

**MLlib ALS requires a Ratings object, which is a tuple in the format (userID, itemID, itemRating)**

In [683]:
# Create Ratings object required for MLlib ALS train function
# Exclude NaN entries for model training; will be used for testing and predicting
# ratings = jdf.flatMap(lambda line: [(line[0], i, line[i]) for i in range(1,101) if np.isnan(line[i])==0])
ratings = jdf.flatMap(lambda line: [(line[0], i, line[i]) for i in range(1,101) if line[i]!=0])

# View snippet of Ratings object
ratings.collect()[:5]

[(u'1', 1, -0.12541000876329006),
 (u'1', 2, 0.1409659817173043),
 (u'1', 3, -0.1549182461193583),
 (u'1', 4, -0.13086261783995484),
 (u'1', 5, -0.12059888310740936)]

**Check the number of users, jokes, and ratings in our dataframe**

In [684]:
numRatings = ratings.count()
numUsers = ratings.map(lambda r: r[0]).distinct().count()
numJokes = ratings.map(lambda r: r[1]).distinct().count()

print "Got %d ratings from %d users on %d jokes." % (numRatings, numUsers, numJokes)

Got 1805072 ratings from 24983 users on 100 jokes.


**Randomly divide ratings into a 75% training and 25% testing set**

In [685]:
ratingsTrain, ratingsTest = ratings.randomSplit([0.75, 0.25], seed = 85)

# Check number of ratings in each set
print "Rated Jokes for All Users"
print "Training set: %d" %(len(ratingsTrain.collect()))
print "Test set: %d" %(len(ratingsTest.collect()))

Rated Jokes for All Users
Training set: 1353794
Test set: 451278


### Train the recommendation model using ALS

In [686]:
from pyspark.mllib.recommendation import ALS, Rating

# Default parameters; can also tune using function defined in MLlib example
rank = 5
numIterations = 10

# Train model with training data
cf_model = ALS.train(ratingsTrain, rank, numIterations)

**Check model accuracy and error**

In [687]:
ratingsTest = sc.parallelize(ratingsTest.collect())
cf_test = cf_model.predictAll(ratingsTest.map(lambda p: (p[0], p[1]))).map(lambda r: ((r[0], r[1]), r[2]))

# [(userID, jokeID), (actual rating, predicted rating)]
cf_results = ratingsTest.map(lambda r: ((int(r[0]), r[1]), r[2])).join(cf_test)
cf_results = sc.parallelize(cf_results.collect())

# Example
cf_results.collect()[:3]

[((5993, 77), (0.0704940933797176, -0.02550462374990562)),
 ((17350, 18), (0.17554646970416904, 0.005708623905954488)),
 ((21952, 22), (0.10988728506862236, 0.07990360094054466))]

In [688]:
MSE = cf_results.map(lambda r: (r[1][0] - r[1][1])**2).mean()
print "Mean Squared Error = %.3f" %(MSE)

RMSE = math.sqrt(MSE)
print "Root Mean Squared Error = %.3f \n" %(RMSE)

MAE = cf_results.map(lambda r: abs((r[1][0] - r[1][1]))).mean()
print "Mean Absolute Error = %.3f" %(MAE)

NMAE = MAE / (10 - -10)
print "Normalized Mean Abolute Error = %.3f" %(NMAE)

Mean Squared Error = 0.009
Root Mean Squared Error = 0.094 

Mean Absolute Error = 0.076
Normalized Mean Abolute Error = 0.004


I tinkered with some of the tuning parameters (rank, iterations) and found the RMSE to be fairly consistent. The ratings scale goes from -10 to 10 (range of 20) so both RMSE and MAE may seem somewhat large. However, in the cited paper above, Goldberg et al. looked at a few other algorithms and found the NMAE to range from 0.187 to 0.237. Relatively, the ALS model here would appear very good. Note that NMAE was preferred by the authors to normalize errors as a percentage of the full scale. 

### Make recommendations using model and set of jokes not rated

In [689]:
def get_rated_notRated(user_num):
    
    jokeID = list(range(1, numJokes + 1))
    userID = str(user_num)

    # User's rated jokes
    uRatings = [(j,r) for (u,j,r) in ratings.collect() if u==userID]
    jokes_rated = [j[0] for j in uRatings]
    jokes_notRated = [j for j in jokeID if j not in jokes_rated]
    
    return (jokes_rated, jokes_notRated)

In [690]:
# Example
user = 1
u1 = get_rated_notRated(user)
print "Rated jokes: %d" %(len(u1[0]))
print "Unrated jokes: %d" %(len(u1[1]))

Rated jokes: 74
Unrated jokes: 26


#### Predicted scores

In [691]:
def get_cf_recs(user, unratedIDs, model):
    
    unratedIDs = sc.parallelize(set(unratedIDs))
    cf_predictions = model.predictAll(unratedIDs.map(lambda x: (user, x))).collect()
    cf_predictions = sorted(cf_predictions, key=lambda k: k[2], reverse=True)
    
    return cf_predictions

In [692]:
# Blank dictionary to hold all recommendations for all users
cf_recs = {}

In [693]:
# Example - top 5 recommendations for User 13
user_num = 1225
user_rated, user_notRated = get_rated_notRated(user_num)

if user_notRated:
    cf_recs[user_num] = get_cf_recs(user_num, user_notRated, cf_model)
    print "Recommended jokes for User %d (score out of 10): \n" %(user_num)
    for i in cf_recs[user_num][:5]:
        print "JokeID: %d    Predicted rating: %.2f" %(i[1], i[2])
else: 
    print "No unrated jokes for User %d" %(user_num)

Recommended jokes for User 1225 (score out of 10): 

JokeID: 89    Predicted rating: 0.02
JokeID: 54    Predicted rating: -0.02
JokeID: 93    Predicted rating: -0.03
JokeID: 76    Predicted rating: -0.03
JokeID: 91    Predicted rating: -0.04


**Print full joke of the top 5 recommendations**

In [694]:
import pickle
jokesFull = pickle.load(open( "jokesFull.p", "rb" ) )

In [695]:
user_num = 1225
for i in cf_recs[user_num][:5]:
    print "Joke #%d:" %(i[1])
    print jokesFull[i[1]] +"\n"

Joke #89:
Q: How many programmers does it take to change a lightbulb? A: NONE! That's a hardware problem. . . .

Joke #54:
A woman has twins,  and gives them up for adoption.  One of them goes to a family in Egypt and is named "Amal. " The other goes to a family in Spain; they name him "Juan. " Years later,  Juan sends a picture of himself to his mom.  Upon receiving the picture,  she tells her husband that she wishes she also had a picture of Amal.  Her husband responds,  "But they are twins-if you've seen Juan,  you've seen Amal.

Joke #93:
Two atoms are walking down the street when one atom says to the other "Oh,  my! I've lost an electron!" The second atom says"Are you sure" The first replies "I'm positive!"

Joke #76:
If pro- is the opposite of con- then congress must be the opposite of progress.

Joke #91:
Early one morning a mother went to her sleeping son and woke him up.  "Wake up,  son.  It's time to go to school. " "But why,  Mama? I don't want to go to school. " "Give me tw

### Summary

Collaborative filtering is interesting because while it does not explicitly account for specific content features about the joke itself (e.g. humour type, general topic), it is able to implicitly deduce such latent factors and be effective at recommending similar items. In this example for User 13, it would appear s/he likes lowbrow humour and relationship topics. The downside of collaborative filtering may be that users must rate their tastes consistently but sometimes this is harder to formalize when the 'item' is more abstract like humour. Additionally, if you equally like many genres or content features and rate several items very similarly, ALS and collaborative filtering may be less successful in its recommendations. This would be true with almost any algorithm though.  

In the Goldberg paper, the authors assessed a few recommendation methods including k-nearest neighbors, global mean, and their proposed Eigentaste algorithm. The latter involves normalizing the ratings and applying principal component analysis (PCA), amongst other steps. They deduced Eigentaste was more appropriate for sparse matrices and 'capturing tastes with finer granularity'. As Eigentaste is effectively a form of collaborative filtering, it should not be too surprising that the ALS method used in this mini-project appears fairly consistent in model error and results.

## Content-Based Recommendations 
### Create Joke (item) Profile

In [696]:
from pyspark.mllib.feature import HashingTF
from pyspark.mllib.linalg import Vectors
from pyspark.mllib.feature import IDF

#### Get corpus of jokes (documents)
* Use cleaned jokes stripped of stopwords and punctuation for better classification

In [697]:
jokesClean = pickle.load(open( "jokesClean.p", "rb" ) )

# Assemble corpus as one file, split by new line
with open('jokesClean.txt', 'w') as jc:
    for k,v in jokesClean.iteritems():
      jc.write("%s\n" % v)

In [698]:
# Load into Spark - should have 100 jokes
documents = sc.textFile("jokesClean.txt").map(lambda line: line.split(" "))
documents.count()

100

#### Feature extraction - use PySpark's tf.idf function  
* add source and explanation

In [699]:
hashingTF = HashingTF()
tf = hashingTF.transform(documents)

tf.cache()
idf = IDF().fit(tf)
tfidf = idf.transform(tf)

joke_features = tfidf.collect()

In [700]:
# Example
jf1 = joke_features[0]
jf1

SparseVector(1048576, {47054: 3.922, 183558: 7.8439, 285677: 3.922, 397847: 2.2172, 419033: 3.0057, 518276: 1.2478, 582313: 3.5165, 605676: 2.4179, 608735: 3.5638, 611077: 2.3125, 640114: 3.2288, 641573: 3.922, 904615: 6.4577, 985518: 3.922})

#### Calculate cosine distance (similarity)

In [749]:
import math
def cosine_distance(v1, v2):
    dotprod = v1.dot(v2)
    sqdist = v1.squared_distance(v2)
    cosine_angle = dotprod / sqdist
    
    try:
        cosine_distance = math.acos(cosine_angle) * (180/math.pi)
    except:
        cosine_distance = 90.0
    
    return np.round(cosine_distance, 2)

In [702]:
# Test using PySpark sparse vectors
sv1 = Vectors.sparse(3, [0,1,2], [1.0,2.0,-1.0])
sv2 = Vectors.sparse(3, [0,1,2], [2.0,1.0,1.0])
print cosine_distance(sv1, sv2) # should be 60

60.0


In [734]:
# Example cosine distance Joke 1 and 2
jf2 = joke_features[1]
print cosine_distance(jf1, jf2)

90.0


In [810]:
for i in range(1, len(joke_features)):
    print cosine_distance(jf1, joke_features[i])

90.0
90.0
88.91
90.0
89.93
90.0
90.0
89.64
89.71
90.0
88.75
90.0
89.91
90.0
90.0
90.0
89.92
90.0
89.05
90.0
89.62
90.0
90.0
90.0
89.57
89.43
89.68
89.85
90.0
89.55
89.03
88.8
90.0
89.28
89.62
89.06
90.0
88.37
90.0
90.0
89.47
89.65
89.68
90.0
89.13
90.0
90.0
90.0
89.46
90.0
90.0
89.21
90.0
90.0
88.89
90.0
90.0
90.0
90.0
89.51
90.0
90.0
90.0
89.51
90.0
90.0
86.89
89.55
90.0
90.0
90.0
90.0
90.0
90.0
88.46
90.0
90.0
90.0
89.78
88.88
89.77
89.89
90.0
90.0
89.24
77.97
85.52
90.0
90.0
90.0
89.48
89.71
89.15
90.0
90.0
89.85
90.0
90.0
90.0


### Create User Profile  

* Weight rating by each joke's features
* Output - sparse vector of user feature rating

In [705]:
# Function to add two sparse vectors together - needed to aggregate user features for all jokes
# http://stackoverflow.com/questions/32981875/how-to-add-two-sparse-vectors-in-spark-using-python

from pyspark.mllib.linalg import SparseVector, DenseVector

def add(v1, v2):
    """Add two sparse vectors
    >>> v1 = Vectors.sparse(3, {0: 1.0, 2: 1.0})
    >>> v2 = Vectors.sparse(3, {1: 1.0})
    >>> add(v1, v2)
    SparseVector(3, {0: 1.0, 1: 1.0, 2: 1.0})
    """
    assert isinstance(v1, SparseVector) and isinstance(v2, SparseVector)
    assert v1.size == v2.size 
    # Compute union of indices
    indices = set(v1.indices).union(set(v2.indices))
    # Not particularly efficient but we are limited by SPARK-10973
    # Create index: value dicts
    v1d = dict(zip(v1.indices, v1.values))
    v2d = dict(zip(v2.indices, v2.values))
    zero = np.float64(0)
    # Create dictionary index: (v1[index] + v2[index])
    values =  {i: v1d.get(i, zero) + v2d.get(i, zero)
       for i in indices
       if v1d.get(i, zero) + v2d.get(i, zero) != zero}

    return Vectors.sparse(v1.size, values)

SparseVector.__add__ = add

In [732]:
# User profile based on rated jokes
def get_user_profile(user_num):
    features = tfidf.collect()
    user_profile = features[0]
    user_ratings = jdf.collect()[user_num - 1]
    for i in range(1, len(features)):
        if user_ratings[i] != 0:
            features[i].values = features[i].values * user_ratings[i+1]
            user_profile = user_profile + features[i]
    return user_profile

### Make recommendations  
* User-Item profiles

In [707]:
# Content-based recommendations
def content_recommend(user_profile, user_notRated):
    
    # Similarity - all jokes
    similarity_all = [{i+1 : cosine_distance(joke_features[i], user_profile)} for i in range(len(jokes))]

    # Similarity - not rated jokes only
    notRated = sc.parallelize(set(user_notRated))
    similarity_notRated = notRated.map(lambda x: (x, similarity_all[x-1][x])).collect()
    similarity_notRated = sorted(similarity_notRated , key=lambda k: k[1])
    return similarity_notRated

In [708]:
# Blank dictionary to hold all recommendations for all users
content_recs = {}

In [750]:
# Example - top 5 recommendations for User 13
user_num = 1225
user_rated, user_notRated = get_rated_notRated(user_num)

if user_notRated:
    up = get_user_profile(user_num)
    content_recs[user_num] = content_recommend(up, user_notRated)
    print "Recommended jokes for User %d: \n" %(user_num)
    for i in content_recs[user_num][:5]:       
        print "JokeID: %d    Similarity distance: %.2f" %(i[0], i[1])
else: 
    print "No unrated jokes for User %d" %(user_num)

Recommended jokes for User 1225: 

JokeID: 87    Similarity distance: 79.77
JokeID: 88    Similarity distance: 86.32
JokeID: 33    Similarity distance: 88.75
JokeID: 39    Similarity distance: 88.96
JokeID: 76    Similarity distance: 89.08


** Recommended jokes**

In [738]:
user_num = 1
for i in content_recs[user_num][:5]:
    print "Joke #%d:" %(i[0])
    print jokes_text[i[0]] +"\n"

Joke #87:
A Czechoslovakian man felt his eyesight was growing steadily worse, and felt it was time to go see an optometrist. The doctor started with some simple testing, and showed him a standard eye chart with letters of diminishing size: CRKBNWXSKZY. . . "Can you read this?" the doctor asked. "Read it?" the Czech answered. "Doc, I know him!"

Joke #88:
A radio conversation of a US naval ship with Canadian authorities ... Americans: Please divert your course 15 degrees to the North to avoid a collision. Canadians: Recommend you divert YOUR course 15 degrees to the South to avoid a collision. Americans: This is the Captain of a US Navy ship. I say again, divert YOUR course. Canadians: No. I say again, you divert YOUR course. Americans: This is the aircraft carrier USS LINCOLN, the second largest ship in the United States' Atlantic Fleet. We are accompanied by three destroyers, three cruisers and numerous support vessels. I demand that you change your course 15 degrees north, that's ONE

## Compare CF and CB

* test set
* pick top N recommendations from CF and CB
* compare with top N from test set
* calculate precision, recall 
* top test into function
* calculate F1
* all in one accuracy function
* redo cf model
* redo user profile
* loop over X users

In [811]:
# Divide ratings into test and training set
user_num = 13
rall = ratings.filter(lambda x: x[0]==str(user_num))

rtrain, rtest = rall.randomSplit([0.75, 0.25], seed = 85)
rtrainIDs = [x[1] for x in rtrain.collect()]
rtestIDs = [x[1] for x in rtest.collect()]

print "Rated Jokes for User %d\n" %(user_num)
print "Total rated: %d" %(rall.count())
print "Training set: %d" %(len(rtrainIDs))
print "Test set: %d" %(len(rtestIDs))

Rated Jokes for User 13

Total rated: 47
Training set: 37
Test set: 10


In [813]:
# Top N recommendations (N = 25% of test set)
n = int(math.ceil(0.25 * len(rtestIDs)))
n

3

#### Actual test set top N

In [814]:
# Get top test set results based on actual ratings
def get_actual_rec(test_set, n):
    test_set = test_set.sortBy(lambda x: x[2], ascending=False)
    test_IDs = [x[1] for x in test_set.collect()]
    top_IDs = [x[1] for x in test_set.collect()[:n]]
    return top_IDs

In [815]:
topTestIDs = get_actual_rec(rtest, n)
print topTestIDs

[49, 36, 93]


#### CF top N

#### TODO: redo training model for rtrainIDs

In [836]:
cf_train_ratings = ratings.filter(lambda x: x[1] not in rtestIDs & x[0]!=user_num)

In [830]:
# Default parameters; can also tune using function defined in MLlib example
rank = 5
numIterations = 10

# Train model with training data
cf_traintest_model = ALS.train(cf_train_ratings, rank, numIterations)

In [838]:
# Get top CF recommendations
compare_cf = get_cf_recs(user_num, rtestIDs, cf_model)
topCFIDs = [x[1] for x in compare_cf[:n]]
print topCFIDs

[36, 49, 42]


#### Content-based top N

In [851]:
# Update user profile function for train/test purposes
# Remove testIDs from user profile since they are equal to 'not rated'
def get_user_profile2(user_num, testIDs):
    features = tfidf.collect()
    user_profile = features[0]
    user_ratings = jdf.collect()[user_num - 1]
    for i in range(1, len(features)):
        if (user_ratings[i] != 0 and i not in testIDs):
            features[i].values = features[i].values * user_ratings[i+1]
            user_profile = user_profile + features[i]
    return user_profile

#### TODO: redo user profile for rTrainIDs

In [852]:
# Get top content-based recommendations
up_comp = get_user_profile2(user_num, rtestIDs)
compare_content = get_content_recs(up_comp, rtestIDs)
topContentIDs = [x[0] for x in compare_content[:n]]
print topContentIDs

[28, 49, 63]


#### Wrap all of the above into one function

In [866]:
def make_traintest_sets(user_num, N):
    
    rall = ratings.filter(lambda x: x[0]==str(user_num))
    rtrain, rtest = rall.randomSplit([0.75, 0.25], seed = 85)
    rtrainIDs = [x[1] for x in rtrain.collect()]
    rtestIDs = [x[1] for x in rtest.collect()]
    
    n = int(math.ceil(N * len(rtestIDs)))
    
    return(rtrain, rtrainIDs, rtest, rtestIDs, n)

def get_top_recommendations(user_num, rtest, rtestIDs, n):
       
    # top Test
    topTestIDs = get_actual_rec(rtest, n)
    
    # top CF
    # TODO: update model
    compare_cf = get_cf_recs(user_num, rtestIDs, cf_model)
    topCFIDs = [x[1] for x in compare_cf[:n]]
    
    # top CB
    # TODO: update user profile
    up_comp = get_user_profile2(user_num, rtestIDs)
    compare_content = get_content_recs(up_comp, rtestIDs)
    topContentIDs = [x[0] for x in compare_content[:n]]
    
    return(topTestIDs, topCFIDs, topContentIDs)

In [854]:
user_num = 122
trainSet, trainIDs, testSet, testIDs, n = make_traintest_sets(user_num)
topTestIDs, topCFIDs, topContentIDs = get_top_recommendations(user_num, testSet, testIDs, n)

print topTestIDs
print topCFIDs
print topContentIDs

[29, 47, 54, 78, 2, 97]
[54, 49, 69, 47, 48, 78]
[47, 29, 56, 69, 54, 97]


### Accuracy Scores

**<a href="http://grouplens.org/site-content/uploads/evaluating-TOIS-20041.pdf">Evaluating collaborative filtering recommender systems (2004)</a>**  
Jonathan L. Herlocker , Joseph A. Konstan , Loren G. Terveen , John T. Riedl  
  
**<a href="http://research.microsoft.com/pubs/115396/EvaluationMetrics.TR.pdf">Evaluating Recommendation Systems</a>**  
Guy Shani and Asela Gunawardana

In [785]:
# Recommendation precision, recallm, F1-score
import sklearn.metrics as sm

def get_rec_accuracy(testRecs, compRecs, numTest, rec_rank=False):
    
    # Recommendation rank does not matter
    if rec_rank:
        
        rec_precision = sm.precision_score(np.array(testRecs), np.array(compRecs), average='macro')
        rec_recall = np.round((rec_precision * len(testRecs)) / float(numTest), 2)
        rec_f1 = np.round((2 * rec_precision * rec_recall) / (rec_precision + rec_recall), 2)   

    # Recommendation rank matters
    else:    
        matches = list(set(testRecs).intersection(compRecs))
        rec_precision = np.round(len(matches) / float(len(testRecs)), 2)
        rec_recall = np.round(len(matches) / float(numTest), 2)
        rec_f1 = np.round((2 * rec_precision * rec_recall) / (rec_precision + rec_recall), 2)
        
    if np.isnan(rec_f1):
        rec_f1 = 0
    
    return (rec_precision, rec_recall, rec_f1)

#### Collaborative filtering accuracy

In [855]:
# Collaborative filtering accuracy
numTest = len(testIDs)
precisionCF, recallCF, f1CF = get_rec_accuracy(topTestIDs, topCFIDs, numTest, rec_rank=False)
print "User %d\n" %user_num
print "Collaborative Filtering Accuracy - Excluding Recommendation Rank"
print "Precision: %.2f" %precisionCF
print "Recall: %.2f" %recallCF
print "F1 score: %.2f\n" %f1CF

precisionCF_rank, recallCF_rank, f1CF_rank = get_rec_accuracy(topTestIDs, topCFIDs, numTest, rec_rank=True)
print "Collaborative Filtering Accuracy - Including Recommendation Rank"
print "Precision: %.2f" %precisionCF_rank
print "Recall: %.2f" %recallCF_rank
print "F1 score: %.2f\n" %f1CF_rank

User 122

Collaborative Filtering Accuracy - Excluding Recommendation Rank
Precision: 0.50
Recall: 0.14
F1 score: 0.22

Collaborative Filtering Accuracy - Including Recommendation Rank
Precision: 0.00
Recall: 0.00
F1 score: 0.00



#### Content-based accuracy

In [856]:
# Content-based accuracy
numTest = len(testIDs)
precisionContent, recallContent, f1Content = get_rec_accuracy(topTestIDs, topContentIDs, numTest, rec_rank=False)
print "User %d\n" %user_num
print "Content-based Accuracy - Excluding Recommendation Rank"
print "Precision: %.2f" %precisionContent
print "Recall: %.2f" %recallContent
print "F1 score: %.2f\n" %f1Content

precisionContent_rank, recallContent_rank, f1Content_rank = get_rec_accuracy(topTestIDs, topContentIDs, numTest, rec_rank=True)
print "Content-based Accuracy - Including Recommendation Rank"
print "Precision: %.2f" %precisionContent_rank
print "Recall: %.2f" %recallContent_rank
print "F1 score: %.2f\n" %f1Content_rank

User 122

Content-based Accuracy - Excluding Recommendation Rank
Precision: 0.67
Recall: 0.18
F1 score: 0.28

Content-based Accuracy - Including Recommendation Rank
Precision: 0.12
Recall: 0.03
F1 score: 0.05



### All users accuracy

In [857]:
# users = range(1,100)
users = range(1, numUsers + 1)
random.shuffle(users)
users = users[:100]

In [867]:
compare_accuracy_50 = pd.DataFrame(index=users, columns=['precision_cf', 'recall_cf', 'f1_cf', 
                                                  'precision_cb', 'recall_cb', 'f1_cb'])

In [868]:
topN = 0.5 #top N percent of recommendations
for u in users:
    trainSet, trainIDs, testSet, testIDs, n = make_traintest_sets(u, topN)
    topTestIDs, topCFIDs, topContentIDs = get_top_recommendations(u, testSet, testIDs, n)
    num_test = len(testIDs)
    pcf, rcf, f1cf = get_rec_accuracy(topTestIDs, topCFIDs, numTest, rec_rank=False)
    pcb, rcb, f1cb = get_rec_accuracy(topTestIDs, topContentIDs, numTest, rec_rank=False)
    
    compare_accuracy_50.ix[u] = np.array([pcf, rcf, f1cf, pcb, rcb, f1cb])

In [869]:
compare_accuracy_50

,precision_cf,recall_cf,f1_cf,precision_cb,recall_cb,f1_cb
9782,0.75,0.14,0.24,0.75,0.14,0.24
3467,0.73,0.36,0.48,0.64,0.32,0.43
7005,0.55,0.27,0.36,1,0.5,0.67
19282,0.56,0.23,0.33,0.78,0.32,0.45
20787,0.73,0.36,0.48,0.91,0.45,0.6
24780,0.45,0.23,0.3,0.82,0.41,0.55
18020,0.67,0.27,0.38,0.78,0.32,0.45
13647,0.5,0.09,0.15,1,0.18,0.31
5095,0.6,0.14,0.23,0.4,0.09,0.15
6767,0.7,0.32,0.44,0.7,0.32,0.44


In [872]:
# Average accuracy scores
compare_accuracy_50.mean(axis=0)

precision_cf    0.6711
recall_cf       0.2553
f1_cf           0.3631
precision_cb    0.7327
recall_cb       0.2817
f1_cb           0.3997
dtype: float64

In [863]:
# Percentage where CF > CB F1 scores
len(compare_accuracy[compare_accuracy.f1_cf > compare_accuracy.f1_cb]) / float(len(compare_accuracy.index))

0.33

In [864]:
len(compare_accuracy[compare_accuracy.f1_cf == compare_accuracy.f1_cb]) / float(len(compare_accuracy.index))

0.3

In [865]:
len(compare_accuracy[compare_accuracy.f1_cf < compare_accuracy.f1_cb]) / float(len(compare_accuracy.index))

0.37

CB better than CF?!

## Clustering

## K-means

In [906]:
# Documentation example
from pyspark.mllib.clustering import KMeans, KMeansModel
from numpy import array
from math import sqrt

data = sc.textFile("/home/brian/workspace/cuny_msda_is622/spark-1.5.1-bin-hadoop2.6/data/mllib/kmeans_data.txt")
parsedData = data.map(lambda line: array([float(x) for x in line.split(' ')]))
# Build the model (cluster the data)
clusters = KMeans.train(parsedData, 2, maxIterations=10,
        runs=10, initializationMode="random")

# Evaluate clustering by computing Within Set Sum of Squared Errors
def error(point):
    center = clusters.centers[clusters.predict(point)]
    return sqrt(sum([x**2 for x in (point - center)]))

WSSSE = parsedData.map(lambda point: error(point)).reduce(lambda x, y: x + y)
print("Within Set Sum of Squared Error = " + str(WSSSE))

Within Set Sum of Squared Error = 0.692820323028


In [899]:
# Code below is k-means on joke features sparse vectors
# TAKES FOREVER TO RUN
# joke_features_par = sc.parallelize(joke_features)

# joke_clusters = KMeans.train(joke_features_par, 5, maxIterations=2,
#         runs=2, initializationMode="random")

### K-means cluster users

In [956]:
jpd_kmeans_user = jpd.iloc[:,1:]
jdf_kmeans_user = sqlc.createDataFrame(jpd_kmeans_user)

In [957]:
from pyspark.mllib.clustering import KMeans, KMeansModel
from numpy import array

k = 10 # number of clusters
user_kmeans = jdf_kmeans_user.map(lambda row: array([x for x in row]))
user_clusters = KMeans.train(user_kmeans, k, maxIterations=10, seed=55)

In [958]:
# Predict cluster of each user
user_kmeans_predict = user_clusters.predict(user_kmeans).take(user_kmeans.count())

In [959]:
# Examine output
print user_kmeans_predict[0]

from collections import Counter
print(Counter(user_kmeans_predict))

0
Counter({5: 4061, 3: 3895, 8: 2717, 1: 2617, 0: 2372, 4: 1982, 9: 1980, 7: 1930, 2: 1720, 6: 1709})


In [960]:
# Sum of squares distance
user_clusters.computeCost(user_kmeans)

16987.183085482116

### K-means cluster jokes

In [961]:
jpd_kmeans_jokes = jpd_kmeans_user.transpose()
jdf_kmeans_jokes = sqlc.createDataFrame(jpd_kmeans_jokes)

In [962]:
k = 5
jokes_kmeans = jdf_kmeans_jokes.map(lambda row: array([x for x in row]))
jokes_clusters = KMeans.train(jokes_kmeans, k, maxIterations=10, seed=55)

In [963]:
# Predict cluster of each user
jokes_kmeans_predict = jokes_clusters.predict(jokes_kmeans).take(jokes_kmeans.count())

In [964]:
# Examine output
print jokes_kmeans_predict[0]
print(Counter(jokes_kmeans_predict))

2
Counter({2: 63, 0: 34, 1: 1, 3: 1, 4: 1})


In [965]:
# Sum of squares distance
jokes_clusters.computeCost(jokes_kmeans)

17584.067350474736

### LDA

In [ ]:
joke_features_par = sc.parallelize(joke_features)

In [966]:
# Documentation example
from pyspark.mllib.clustering import LDA, LDAModel
from pyspark.mllib.linalg import Vectors

# Load and parse the data
data = sc.textFile("/home/brian/workspace/cuny_msda_is622/spark-1.5.1-bin-hadoop2.6/data/mllib/sample_lda_data.txt")
parsedData = data.map(lambda line: Vectors.dense([float(x) for x in line.strip().split(' ')]))
# Index documents with unique IDs
corpus = parsedData.zipWithIndex().map(lambda x: [x[1], x[0]]).cache()

# Cluster the documents into three topics using LDA
ldaModel = LDA.train(corpus, k=3)

# Output topics. Each is a distribution over words (matching word count vectors)
print("Learned topics (as distributions over vocab of " + str(ldaModel.vocabSize()) + " words):")
topics = ldaModel.topicsMatrix()
for topic in range(3):
    print("Topic " + str(topic) + ":")
    for word in range(0, ldaModel.vocabSize()):
        print(" " + str(topics[word][topic]))

Learned topics (as distributions over vocab of 11 words):
Topic 0:
 6.06242933367
 6.63111741428
 4.41304546174
 32.6199586258
 3.95014154544
 2.64793546749
 4.3003806703
 0.857599216182
 1.68984316352
 8.65447369017
 13.8611214153
Topic 1:
 7.75583529139
 7.50220399212
 2.53054294983
 4.6897212649
 3.02407120052
 5.36259463874
 24.5954891441
 1.34266914869
 4.97476762903
 7.49788766678
 16.5623160588
Topic 2:
 12.1817353749
 14.8666785936
 5.05641158843
 2.69032010931
 18.025787254
 13.9894698938
 2.10413018563
 7.79973163513
 1.33538920746
 7.84763864306
 2.57656252588


In [976]:
# LDA for joke features

from pyspark.mllib.clustering import LDA, LDAModel
from pyspark.mllib.linalg import Vectors

# Load and parse the data
# data = sc.textFile("/home/brian/workspace/cuny_msda_is622/spark-1.5.1-bin-hadoop2.6/data/mllib/sample_lda_data.txt")
# parsedData = data.map(lambda line: Vectors.dense([float(x) for x in line.strip().split(' ')]))
# Index documents with unique IDs

corpus = joke_features.zipWithIndex().map(lambda x: [x[1], x[0]]).cache()

# Cluster the documents into three topics using LDA
ldaModel = LDA.train(corpus, k=3)

# Output topics. Each is a distribution over words (matching word count vectors)
print("Learned topics (as distributions over vocab of " + str(ldaModel.vocabSize()) + " words):")
topics = ldaModel.topicsMatrix()
for topic in range(3):
    print("Topic " + str(topic) + ":")
    for word in range(0, ldaModel.vocabSize()):
        print(" " + str(topics[word][topic]))

AttributeError: 'list' object has no attribute 'zipWithIndex'